# Анализ базы данных - SQL

**Задачи**

- Посчитать, сколько книг вышло после 1 января 2000 года;
- Для каждой книги посчитать количество обзоров и среднюю оценку;
- Определить издательство, которое выпустило наибольшее число книг толще 50 страниц — чтобы исключить из анализа брошюры;
- Определить автора с самой высокой средней оценкой книг — учитывать только книги с 50 и более оценками;
- Посчитать среднее количество обзоров от пользователей, которые поставили больше 50 оценок.

## 1. Обзор данных

In [1]:
import pandas as pd
from sqlalchemy import create_engine 

In [2]:
db_config = {'user': 'praktikum_student',
 'pwd': 'Sdf4$2;d-d30pp',
 'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
 'port': 6432, 
 'db': 'data-analyst-final-project-db'} 
connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
 db_config['pwd'],
 db_config['host'],
 db_config['port'],
 db_config['db']) 

engine = create_engine(connection_string, connect_args={'sslmode':'require'}) 

### 1.1 Просмотр данных

In [3]:
def sql_query(query):
    df = pd.io.sql.read_sql(query, con = engine)
    return df

In [4]:
books = 'SELECT * FROM books'
sql_query(books).head()

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


In [5]:
sql_query(books).info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   book_id           1000 non-null   int64 
 1   author_id         1000 non-null   int64 
 2   title             1000 non-null   object
 3   num_pages         1000 non-null   int64 
 4   publication_date  1000 non-null   object
 5   publisher_id      1000 non-null   int64 
dtypes: int64(4), object(2)
memory usage: 47.0+ KB


In [6]:
authors = 'SELECT * FROM authors'
sql_query(authors).head()

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


In [7]:
sql_query(authors).info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 636 entries, 0 to 635
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   author_id  636 non-null    int64 
 1   author     636 non-null    object
dtypes: int64(1), object(1)
memory usage: 10.1+ KB


In [8]:
publishers = 'SELECT * FROM publishers'
sql_query(publishers).head()

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


In [9]:
sql_query(publishers).info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 340 entries, 0 to 339
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   publisher_id  340 non-null    int64 
 1   publisher     340 non-null    object
dtypes: int64(1), object(1)
memory usage: 5.4+ KB


In [10]:
ratings = 'SELECT * FROM ratings'
sql_query(ratings).head()

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


In [11]:
sql_query(ratings).info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6456 entries, 0 to 6455
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   rating_id  6456 non-null   int64 
 1   book_id    6456 non-null   int64 
 2   username   6456 non-null   object
 3   rating     6456 non-null   int64 
dtypes: int64(3), object(1)
memory usage: 201.9+ KB


In [12]:
reviews = 'SELECT * FROM reviews'
sql_query(reviews).head()

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


In [13]:
sql_query(reviews).info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2793 entries, 0 to 2792
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   review_id  2793 non-null   int64 
 1   book_id    2793 non-null   int64 
 2   username   2793 non-null   object
 3   text       2793 non-null   object
dtypes: int64(2), object(2)
memory usage: 87.4+ KB


## 2. Задачи

### 2.1 Количество книг вышедших после 1 января 2000 года

In [14]:
books_count_from_jan = """ 
SELECT 
    COUNT(*)
FROM books
WHERE publication_date::date >= '01-01-2000'
"""
book_cnt = sql_query(books_count_from_jan)
book_cnt

,count
0,821


+ С 1 января 2000 года вышла 821 книга

### 2.2 Количество обзоров и средняя оценка каждой книги

In [15]:
ratings = '''
    SELECT books.book_id, title, AVG(rating), COUNT(DISTINCT review_id) FROM books 
    LEFT JOIN ratings ON books.book_id = ratings.book_id
    LEFT JOIN reviews ON books.book_id = reviews.book_id
    GROUP BY books.book_id
'''

books_ratings = sql_query(ratings)
books_ratings

,book_id,title,avg,count
0,1,'Salem's Lot,3.666667,2
1,2,1 000 Places to See Before You Die,2.500000,1
2,3,13 Little Blue Envelopes (Little Blue Envelope...,4.666667,3
3,4,1491: New Revelations of the Americas Before C...,4.500000,2
4,5,1776,4.000000,4
...,...,...,...,...
995,996,Wyrd Sisters (Discworld #6; Witches #2),3.666667,3
996,997,Xenocide (Ender's Saga #3),3.400000,3
997,998,Year of Wonders,3.200000,4
998,999,You Suck (A Love Story #2),4.500000,2


### 2.3 Издательство, выпустившее наибольшее число книг толще 50 страниц

In [16]:
max_books = """
SELECT 
        subquery_A.publisher_id,
        subquery_B.publisher,
        subquery_A.books_cnt
        

FROM 
(SELECT 
    publisher_id AS publisher_id,
    COUNT(book_id) AS books_cnt
FROM books
WHERE num_pages > 50
GROUP BY publisher_id
ORDER BY books_cnt DESC
LIMIT 1) AS subquery_A

LEFT JOIN 

(SELECT 
    publisher AS publisher,
    publisher_id AS publisher_id
FROM publishers) AS subquery_B ON subquery_A.publisher_id = subquery_B.publisher_id

"""
publisher_max_books = sql_query(max_books)
publisher_max_books

,publisher_id,publisher,books_cnt
0,212,Penguin Books,42


+ Издательство Penguin Books выпустило 42 книги

### 2.4 Автор с самой высокой средней оценкой книг

In [17]:
author = """
SELECT subtable.author AS author,
        AVG(subtable.rating) AS rating_avg
        

FROM
(SELECT 
    books_ratings_2.book_id, books_ratings_2.rating_id, books_ratings_2.rating, books_ratings_2.author_id, authors_names.author
    FROM
(SELECT 
    books_ratings.book_id, books_ratings.rating_id, books_ratings.rating, authors_query.author_id
    FROM (SELECT 
        books_more_than_50.book_id,
        sub_ratings.rating_id,
        sub_ratings.rating

FROM
(SELECT 
    book_id AS book_id,
    AVG(rating) AS rating_avg,
    COUNT(rating_id) AS rating_cnt    
FROM ratings
GROUP BY book_id
HAVING COUNT(rating_id) > 50) AS books_more_than_50

LEFT JOIN
(SELECT * FROM ratings) AS sub_ratings ON sub_ratings.book_id = books_more_than_50.book_id) AS books_ratings

LEFT JOIN (SELECT book_id AS book_id, author_id AS author_id FROM books) AS authors_query 
    ON authors_query.book_id = books_ratings.book_id) AS books_ratings_2

LEFT JOIN (SELECT * FROM authors) AS authors_names ON books_ratings_2.author_id = authors_names.author_id) AS subtable

GROUP BY subtable.author
ORDER BY rating_avg DESC


"""
books_more_than_50 = sql_query(author)
books_more_than_50

,author,rating_avg
0,J.K. Rowling/Mary GrandPré,4.287097
1,Markus Zusak/Cao Xuân Việt Khương,4.264151
2,J.R.R. Tolkien,4.246914
3,Louisa May Alcott,4.192308
4,Rick Riordan,4.080645
5,William Golding,3.901408
6,J.D. Salinger,3.825581
7,Paulo Coelho/Alan R. Clarke/Özdemir İnce,3.789474
8,William Shakespeare/Paul Werstine/Barbara A. M...,3.787879
9,Lois Lowry,3.750000


+ Автором с самой высокой оценкой (4.3) является J.K. Rowling

### 2.5 Среднее количество обзоров от пользователей

In [18]:
avg_user = """ 
SELECT AVG(review_cnt) AS avg_reviews FROM

(SELECT 
        username AS username,
        COUNT(review_id) AS review_cnt

FROM
(SELECT 
        users_more_than_50.username,
        reviews_data.review_id

FROM (SELECT 
    username AS username,
    COUNT(rating_id) AS rating_cnt
FROM ratings
GROUP BY username
HAVING COUNT(rating_id) > 50) AS users_more_than_50

LEFT JOIN 
    (SELECT 
            username AS username,
            review_id AS review_id
    FROM reviews
    ) AS reviews_data ON reviews_data.username  = users_more_than_50.username) AS subtable
    
GROUP BY username) AS avg_cnt

"""
avg_cnt = sql_query(avg_user)
avg_cnt

,avg_reviews
0,24.333333


+ В среднем пользователи оставившие больше 50 оценок пишут к половине из них отзыв

## 3. Итоги

+ С 1 января 2000 года вышла 821 книга.
+ Таблица с количеством обзор и средним рейтингом 'reviews_rating'.
+ Издательством, выпустившем наибольшее количество книг является Penguin Books (42 книги).
+ Автором с самой высокой оценкой является J.K. Rowling (рейтинг 4.3).
+ В среднем пользователи оставившие больше 50 оценок пишут к половине из них отзыв (24 отзыва).